In [32]:
import os
import cv2
import numpy as np
from PIL import Image
from enum import Enum
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.signal import savgol_filter

In [44]:
def zero_pad(Y):
    mx, my = Y.shape
    padded_x = ((mx + 7) // 8) * 8
    padded_y = ((my + 7) // 8) * 8
    img = np.zeros((padded_x, padded_y), dtype=np.uint8)
    img[:mx, :my] = Y
    
    return img

def DCT(img):
    w, h = img.shape
    num_blocks = (w // 8) * (h // 8)
    dct_output = np.zeros((8, 8, num_blocks), dtype=float) 

    for ci, (i, j) in enumerate((x, y) for x in range(0, w, 8) for y in range(0, h, 8)):
        dct_output[:, :, ci] = cv2.dct(img[i:i+8, j:j+8].astype(np.float32))  # Ensure input is float for DCT

    return dct_output


def hist(DC, N=2000, show=False):
    hcount, bin_edges = np.histogram(DC, bins=N, density=True)
    if show:
        plt.figure(figsize=(10, 6))
        sns.histplot(x=bin_edges[:-1], weights=hcount, bins=N, kde=True, stat="density")
        plt.title("DC Coefficient Histogram")
        plt.xlabel("DC Coefficient Value")
        plt.ylabel("Density")
        plt.show()
    return hcount, bin_edges

def FFT(hcount):
  hcount -= np.mean(hcount)
  x = np.fft.fft(hcount.astype(float))
  x = abs(np.fft.fftshift(x))
  x = savgol_filter(x, 11, 2)
  return x

def quality(x, N=2000):
    ref = x[N // 2 - 1]
    candidate_indices = np.where((x - ref) > -12)[0]

    local_maxs = [
        i for i in candidate_indices
        if 3 <= i < x.shape[0] - 3  
        and x[i] > x[i - 1] and x[i] > x[i + 1]
        and x[i] > x[i - 3] + 1 and x[i] > x[i + 3] + 1
    ]

    if len(local_maxs) <= 1:
        Q = 100
    else:
        distances = np.diff(local_maxs)
        average_distance = np.mean(distances)


        max_distance = 40   
        min_distance = 5   

        Q = 100 - (average_distance - min_distance) / (max_distance - min_distance) * 90
        Q = max(0, min(100, Q))  

    print("Average Distance:", average_distance if len(local_maxs) > 1 else "N/A")
    return round(Q), local_maxs


def find_quality_level(x, N=2000):
  # Find distance between peaks in FFT
  K=0
  Q=0
  # Use a reference magnitude
  ref = x[N//2-1]

  # Find locations that are 12 db lower than max magn.
  r=np.where((x-ref)>-12)

  local_maxs=[]
  for i in r[0]:
    if i<3 or i>x.shape[0]-3: # ignore the edges of FFT
      continue

    # Find local peaks that have snr greater than 50% of the highest peak
    if x[i] > x[i-1] and x[i] > x[i+1] and x[i] > x[i-3] + 0.01 * np.max(x) and x[i] > x[i+3] + 0.5 * np.max(x):
      local_maxs.append(i)

  # Use lookup table and get the compression level
  K_table = np.array([10, 50, 100]) # This is the lookup table that I extracted using non-compressed image set
  if len(local_maxs)==1: # if no local max
    Q=100
  else:
    # Select the most frequent distance between peaks
    d = np.bincount(np.diff(local_maxs)).argmax()
    K = d/N*100

    # Use lookup table, interpolate and find compression
    for i in range(9):
      if K>=K_table[i] and K<K_table[i+1]:
        Q=(i+1)*10+(K-K_table[i])/(K_table[i+1]-K_table[i])*10

  print("K:", K)
  return round(Q), local_maxs

In [45]:
img = Image.open("0010x8.png")

img = img.convert('YCbCr')
img = zero_pad(np.array(img)[:,:,0])
img = img.astype(np.float64)
dct = DCT(img)[0,0,:]
hcount, bin_edges = hist(dct)
fft = FFT(hcount)
image_quality, max_list = find_quality_level(fft)
print(max_list)

print(f"Quality of the image: {image_quality}")

fig = plt.figure(figsize=(20,8))
plt.plot(hcount)
fig.suptitle("Histogram of DC Term",fontsize=20)


plt.figure(figsize=(20, 8))
plt.plot(fft)
plt.plot(max_list, fft[max_list], marker='s')
plt.title("FFT of DC Coefficient Histogram")
plt.xlabel("Frequency")
plt.ylabel("Magnitude (dB)")
plt.show()

TypeError: Cannot cast array data from dtype('float64') to dtype('int64') according to the rule 'safe'